In [75]:
# Importation des modules

import pandas as pd
import nltk 
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re


from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='french')

#Affichage de toutes les colonnes

pd.set_option('display.max_columns', 500)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lilianmarey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
#Importation des données

df = pd.read_csv('data/bdd_complete2.csv', sep = ',', encoding = 'latin-1').sample(50000)

In [77]:
# Echantillon de la base
print(df.shape)
df.sample(5)

(50000, 26)


,Unnamed: 0,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor
448845,448846,stephane peu,2019-12-11,Violences au sein de la famille,Présidence de M. Sylvain Waserman,Discussion générale,De même laccès facilité au téléphone grave d...,NaN,Présidence de M. Sylvain Waserman,Peu,M,55,GDR,Lois,0,True,23,3 - Ancr. local,3 - Ancr. local,PAS D'INFO,1962,False,collab,448846,Exp,2019-12-01
166754,166755,francis vercamer,2017-10-26,Projet de loi de financement de la Sécurité so...,Présidence de Mme Danielle Brulebois,NaN,Alors que nous débattons des recettes du PLFSS...,NaN,Présidence de Mme Danielle Brulebois,Vercamer,M,59,LC,Aff. Sociales,3,False,28,5 - Dép. sortant,5 - Dép. sortant,PtEc,1958,True,cadsupprive,166755,Exp,2017-10-01
277238,277239,joel giraud,2018-10-22,Projet de loi de finances pour 2019,Présidence de M. Maurice Leroy,NaN,Pour autant nous devons mieux coordonner nos ...,NaN,Présidence de M. Maurice Leroy,Giraud,M,58,LREM,Finances,3,False,28,5 - Dép. sortant,5 - Dép. sortant,Master,1959,True,cadsuppub,277239,Exp,2018-10-01
372226,372227,philippe berta,2017-12-13,Orientation et réussite des étudiants,Présidence de M. François de Rugy,Discussion des articles,Je suis tout à fait daccord avec ce qui a ét...,NaN,Présidence de M. François de Rugy,Berta,M,57,MODEM,Aff. Cult. & Educ.,0,False,0,1 - Soc. civile,6  Exp. Pol.,Doct,1960,True,cadsuppub,372227,Exp,2017-12-01
287304,287305,julien dive,2018-05-24,Équilibre dans le secteur agricole et alimentaire,Présidence de M. de Rugy,Discussion des articles,Compte tenu de la théorie du ruissellement qu...,NaN,Présidence de M. de Rugy,Dive,M,32,LR,affaires économiques,1,False,7,5 - Dép. sortant,5 - Dép. sortant,Master,1985,False,cadsupprive,287305,Exp,2018-05-01


In [78]:
# L'objectif est de nettoyer le texte dit par les intervenants. On s'intéresse donc aux colonnes 'Réplique' et 'Didascalie'

df_texte = df[['Réplique', 'Didascalie']]

In [79]:
# Première correction : les apostrophes sont codées par '\x92'. On les remplace par des espaces. 
# On supprime aussi les codes '<U+0080>' qui sont avant les points d'exclamation
# On remplace les tirets par des espaces
# On met tout en minuscule

df_texte['Réplique'] = df_texte.apply(
    lambda row: row.Réplique.replace('\x92', ' ').replace('<U+0080>', '').replace('-', ' ').lower(),
    axis = 1)

df_texte['Didascalie'] = df_texte.apply(
    lambda row: str(row.Didascalie).replace('\x92', ' ').replace('<U+0080>', '').replace('-', ' ').lower(),
    axis = 1)


/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [80]:
# Séparation des didascalies

def extract_didascalie(str):
    li=[]
    result = re.search(r'\((.*?)\)',str)
    if result !=None:
        n=len(result.groups())
        for i in range(1,n+1):
            a=''.join(result.groups(i))
            a=a.replace('(','')
            a=a.replace(',','')
            a=a.replace(')','')
            a=a.replace("'",'')
            li.append(a)
    return li

def extract_replique(str):
    rep=str
    li=extract_didascalie(str)
    for i in li:
        rep=rep.replace(i,"")
        
    rep=rep.replace("()","")   
    return rep

df_texte['Réplique'] = df_texte.apply(lambda row: extract_replique(row.Réplique), axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [81]:
# On tokenize les chaines de caractère

df_texte['tokenized_replique'] = df_texte.apply(lambda row: nltk.word_tokenize(row.Réplique, language='french'), 
                                             axis = 1)
df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: nltk.word_tokenize(row.Didascalie, language='french'), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [82]:
# On obtient bien des listes de mots

df_texte.head(10)

# On remplace les nan par des listes vides

df_texte['tokenized_replique'] = df_texte.apply(lambda row: row.tokenized_replique if len(row.tokenized_replique) == 0 or row.tokenized_replique[0] != 'nan' else [], axis = 1)
df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: row.tokenized_didascalie if len(row.tokenized_didascalie) == 0 or row.tokenized_didascalie[0] != 'nan' else [], axis = 1)



/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [83]:
# On retire la ponctuation pour le moment

df_texte['tokenized_replique'] = df_texte.apply(lambda row: [word for word in row.tokenized_replique if word.isalpha()], 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: [word for word in row.tokenized_didascalie if word.isalpha()], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [84]:
# On retire les stopwords

stop_words = set(stopwords.words('french'))

df_texte['tokenized_replique'] = df_texte.apply(lambda row: [word for word in row.tokenized_replique if not word in stop_words], 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: [word for word in row.tokenized_didascalie if not word in stop_words], 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [85]:
# Stemming

df_texte['stemmed_replique'] = df_texte.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.tokenized_replique]), 
                                             axis = 1)

df_texte['stemmed_didascalie'] = df_texte.apply(lambda row: ' '.join([stemmer.stem(word) for word in row.tokenized_didascalie]), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [86]:
# On repasse les listes en chaines de caractère


df_texte['tokenized_replique'] = df_texte.apply(lambda row: ' '.join(row.tokenized_replique), 
                                             axis = 1)

df_texte['tokenized_didascalie'] = df_texte.apply(lambda row: ' '.join(row.tokenized_didascalie), 
                                             axis = 1)

/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/lilianmarey/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
df_cleaned = df.copy()
df_cleaned['tokenized_replique'] = df_texte['tokenized_replique']
df_cleaned['tokenized_didascalie'] = df_texte['tokenized_didascalie']
df_cleaned['stemmed_replique'] = df_texte['stemmed_replique']
df_cleaned['stemmed_didascalie'] = df_texte['stemmed_didascalie']

In [88]:
df_cleaned.to_csv('data/data_cleaned_test.csv', sep = ',', encoding = 'latin-1')

In [90]:
pd.read_csv('data/data_cleaned_test.csv', sep = ',', encoding = 'latin-1').sample(10)

,Unnamed: 0,Unnamed: 0.1,Orateur,Date,Nature.de.séance,Président.de.séance,Sujet.débattu,Réplique,Didascalie,Président.de.séance_propre,nom.famille,sexe,age,groupe.sigle,commissions,nb.mandats,cabcollab,duree.pol,clustRFSP,clustVEP,hautdip,naissance_an,majo,profsigni2,ID,Groupe,time_floor,tokenized_replique,tokenized_didascalie,stemmed_replique,stemmed_didascalie
20821,389159,389160,pierre dharreville,2019-10-22,Projet de loi de financement de la sécurité so...,Présidence de M. Richard Ferrand,NaN,Si nous sommes confrontés à des non-compensati...,(Applaudissements sur plusieurs bancs du grou...,Présidence de M. Richard Ferrand,Dharréville,M,42,GDR,Aff. Sociales,0,True,10,4 - Entourage,4 - Entourage,Licence,1975,False,cadsupprive,389160,Exp,2019-10-01,si confrontés non compensations parce exonérat...,applaudissements plusieurs bancs groupe fi,si confront non compens parc exoner massiv acc...,applaud plusieur banc group fi
41008,263840,263841,jean paul lecoq,2018-07-23,Liberté de choisir son avenir professionnel,Présidence de M. François de Rugy,Discussion des articles,Au moins cela figurera au compte rendu des d...,NaN,Présidence de M. François de Rugy,Lecoq,M,59,GDR,Aff. Etrangeres,1,False,28,5 - Dép. sortant,5 - Dép. sortant,Pro,1958,False,EmployeInter,263841,Exp,2018-07-01,moins cela figurera compte rendu débats,NaN,moin cel figur compt rendu débat,NaN
46018,427684,427685,said ahamada,2018-10-18,Projet de loi de finances pour 2019,Présidence de M. Sylvain Waserman,NaN,Jai lu létude dimpact avec attention et j...,NaN,Présidence de M. Sylvain Waserman,Ahamada,M,45,LREM,Finances,0,False,3,1 - Soc. civile,6  Exp. Pol.,Master,1972,True,cadsuppub,427685,Exp,2018-10-01,lu étude impact attention vu comme cette mesur...,NaN,lu étud impact attent vu comm cet mesur aid in...,NaN
40593,270017,270018,jean rene cazeneuve,2020-02-25,Système universel de retraite,Présidence de M. Richard Ferrand,Discussion des articles,Cest faux<U+0080>!,NaN,Présidence de M. Richard Ferrand,Cazeneuve,M,59,LREM,Finances,0,False,3,1 - Soc. civile,6  Exp. Pol.,GdEc,1958,True,cadsupprive,270018,Exp,2020-02-01,faux,NaN,faux,NaN
18661,366647,366648,patrick mignola,2019-05-09,Droit voisin au profit des agences et éditeurs...,Présidence de M. Hugues Renson,Discussion des articles,Dans lhypothèse où lon ferait démarrer une n...,NaN,Présidence de M. Hugues Renson,Mignola,M,46,MODEM,Aff. Sociales,0,True,21,3 - Ancr. local,3 - Ancr. local,ScPo,1971,True,collab,366648,Exp,2019-05-01,hypothèse où ferait démarrer nouvelle période ...,NaN,hypothes où fer démarr nouvel périod protect t...,NaN
17562,366715,366716,patrick mignola,2020-02-25,Système universel de retraite,Présidence de M. Richard Ferrand,Discussion des articles,Je crois que les masques tombent. (Applaudiss...,(Applaudissements sur les bancs du groupe MOD...,Présidence de M. Richard Ferrand,Mignola,M,46,MODEM,Aff. Sociales,0,True,21,3 - Ancr. local,3 - Ancr. local,ScPo,1971,True,collab,366716,Exp,2020-02-01,crois masques tombent,applaudissements bancs groupe modem,crois masqu tombent,applaud banc group modem
3623,138691,138692,eric coquerel,2020-04-17,Projet de loi de finances rectificative pour 2020,Présidence de M. Richard Ferrand,Discussion générale,Les solutions pour demain ne sont pas à trouve...,NaN,Présidence de M. Richard Ferrand,Coquerel,M,59,LFI,Finances,0,False,7,1 - Soc. civile,6  Exp. Pol.,PAS D'INFO,1958,False,AgriArtiCommPDG,138692,Exp,2020-04-01,solutions demain trouver ni chercher toujours ...,NaN,solut demain trouv ni cherch toujour sous yeux...,NaN
3546,216057,216058,gwendal rouillard,2019-12-10,Questions orales sans débat,Présidence de Mme Laetitia Saint-Paul,Conséquences du Brexit sur la pêche bretonne,Jarrête ici de donner des chiffres mais chac...,NaN,Présidence de Mme Laetitia Saint-Paul,Rouillard,M,41,LREM,Defense,2,True,16,5 - Dép. sortant,5 - Dép. sortant,Master,1976,True,collab,216058,Exp,2019-12-01,arrête ici donner chiffres chacun bien compris...,NaN,arrêt ici don chiffr 